<a href="https://colab.research.google.com/github/IzhbuldinaDaria2002/-/blob/main/%D0%A2%D0%B5%D1%81%D1%82%D0%BE%D0%B2%D0%BE%D0%B5_%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

# Загрузка
df_orders = pd.read_csv('web_app_orders_traffic_all.csv')
df_app_sessions = pd.read_csv('app_traffic_all.csv')

# Web-сессии
web_sessions = df_orders[df_orders['platform'] == 'web'][
    ['date', 'country', 'channel_1', 'channel_2', 'sessions']
].copy()
web_sessions.rename(columns={'sessions': 'web_sessions'}, inplace=True)

# Заказы и доход (суммарно)
orders_revenue = df_orders.groupby(
    ['date', 'country', 'channel_1', 'channel_2'],
    as_index=False
).agg({
    'orders': 'sum',
    'revenue': 'sum'
})

# App-сессии
app_sessions = df_app_sessions[['date', 'country', 'channel_1', 'channel_2', 'sessions']].copy()
app_sessions.rename(columns={'sessions': 'app_sessions'}, inplace=True)

# Объединение
merged = pd.merge(orders_revenue, web_sessions, on=['date', 'country', 'channel_1', 'channel_2'], how='outer')
merged = pd.merge(merged, app_sessions, on=['date', 'country', 'channel_1', 'channel_2'], how='outer')

# Заполнение NaN
merged.fillna({'web_sessions': 0, 'app_sessions': 0, 'orders': 0, 'revenue': 0}, inplace=True)

# Общие сессии
merged['sessions'] = merged['web_sessions'] + merged['app_sessions']

# Сохранение
merged.to_csv('merged_data.csv', index=False)

In [3]:
import pandas as pd

df = pd.read_csv('merged_data.csv')
df['date'] = pd.to_datetime(df['date'])

# Фильтр по январю
df_jan = df[df['date'].dt.month == 1]

# Добавляем колонку с годом
df_jan['year'] = df_jan['date'].dt.year

# Группируем по году, стране и каналам
grouped = df_jan.groupby(['year', 'country', 'channel_1', 'channel_2']).agg({
    'sessions': 'sum',
    'orders': 'sum',
    'revenue': 'sum'
}).reset_index()

# CR и AOV
grouped['cr'] = grouped['orders'] / grouped['sessions']
grouped['aov'] = grouped['revenue'] / grouped['orders']

# Разделяем на 2024 и 2025
g2024 = grouped[grouped['year'] == 2024].copy()
g2025 = grouped[grouped['year'] == 2025].copy()

# Объединяем для расчёта дельты
merged_delta = pd.merge(
    g2024, g2025,
    on=['country', 'channel_1', 'channel_2'],
    suffixes=('_2024', '_2025'),
    how='outer'
).fillna(0)

# Дельта в процентах
for metric in ['sessions', 'orders', 'revenue', 'cr', 'aov']:
    col_2024 = f'{metric}_2024'
    col_2025 = f'{metric}_2025'
    merged_delta[f'delta_{metric}'] = (merged_delta[col_2025] - merged_delta[col_2024]) / merged_delta[col_2024] * 100

# Сохраняем результат
merged_delta.to_csv('delta_by_channels.csv', index=False)